In [10]:
import pulp

lp_problem = pulp.LpProblem("Minimize_Total_Cost", pulp.LpMinimize)



In [11]:
generators = ["Type1", "Type2", "Type3"]
#hydro = ["Type1", "Type2"]
periods = ["Period1", "Period2", "Period3", "Period4", "Period5"]


periods_hours = [6, 3, 6, 3, 6]
num_generators = {"Type1": 12, "Type2": 10, "Type3": 5}
costs_above_min = {"Type1": 2, "Type2": 1.30, "Type3": 3}
costs_min = {"Type1": 1000, "Type2": 2600, "Type3": 3000}
startup_costs = {"Type1": 2000, "Type2": 1000, "Type3": 500}
generator_limits = {"Type1": (850, 2000), "Type2": (1250, 1750), "Type3": (1500, 4000)}
load_demands = {"Period1": 15000, "Period2": 30000, "Period3": 25000, "Period4": 40000, "Period5": 27000}

# hyOpLevel = {'type1' : 900, 'type2' : 1400}  # Li is the operating level of hydro i.
# hyCostPerHour = {'type1' : 90, 'type2' : 150}
# hyStartCost = {'type1' : 1500, 'type2' : 1200}
# hyDepthReductionPerHour = {'type1' : 0.31, 'type2' : 0.47}



n = pulp.LpVariable.dicts("Count", (generators, periods), lowBound=0, cat=pulp.LpInteger)
s = pulp.LpVariable.dicts("StartUp", (generators, periods), lowBound=0, cat=pulp.LpInteger)
x = pulp.LpVariable.dicts("ElectricityProduced", (generators, periods), lowBound=0, cat="Continuous")
# h = pulp.LpVariable.dicts("Hydro", (hydro, periods), lowBound=0, cat=pulp.LpInteger)
# t = pulp.LpVariable.dicts("HydroStart", (generators, periods), lowBound=0, cat=pulp.LpBinary)
# l = pulp.LpVariable.dicts("ReserviorHeight", (periods), lowBound=0, cat="Continuous")
# p = pulp.LpVariable.dicts("PumpingCount", (periods), lowBound=0, cat=pulp.LpInteger)





In [12]:


#Demand must be met in each period:
for per in periods:
    lp_problem += pulp.lpSum(x[gen][per] for gen in generators) >= load_demands[per], f"LoadDemand_{per}"
 

In [13]:
   
#Output must lie within the limits of the generators working:
for gen in generators:
    for per in periods:
        lp_problem += x[gen][per] >= n[gen][per] * generator_limits[gen][0] , f"mn_{gen}_{per}"
        lp_problem += x[gen][per] <= n[gen][per] * generator_limits[gen][1] , f"mx_{gen}_{per}"
#The extra guaranteed load requirement must be able to be met without starting up any more generators:
reserve_factor = 1.15
for per in periods:
    lp_problem += pulp.lpSum(n[gen][per] * generator_limits[gen][1] for gen in generators) >= reserve_factor * load_demands[per], f"ReseverDemand_{per}"

#The number of generators started in period j must equal the increase in number:
for gen in generators:

    for j in range(len(periods)):
        per = periods[j]
        prv_per = periods[(j-1)%len(periods)]
        lp_problem += s[gen][per] >= n[gen][per] - n[gen][prv_per], f"NumStartedGenerators_{gen}_{per}"


In [14]:
   
#In addition all the integer variables have simple upper bounds corresponding to the total number of generators of each type.
for gen in generators:
    for j in range(len(periods)):
        lp_problem += n[gen][periods[j]] <= num_generators[gen], f"NumGenerators_{gen}_{periods[j]}"
        lp_problem += s[gen][periods[j]] <= num_generators[gen]-n[gen][periods[(j-1)%len(periods)]], f"StartedGenerators_{gen}_{periods[j]}"


#Objective Function (to be Minimized)
#creat a simplevariable 
firstPart = pulp.lpSum(costs_above_min[gen] * periods_hours[periods.index(per)] * (x[gen][per] - generator_limits[gen][0] * n[gen][per]) for gen in generators for per in periods)
secondPart = pulp.lpSum(costs_min[gen] * n[gen][per] * periods_hours[periods.index(per)] for gen in generators for per in periods)
thirdPart = pulp.lpSum(startup_costs[gen] * s[gen][per] for gen in generators for per in periods)

lp_problem += firstPart + secondPart + thirdPart, "TotalCost"
# Solve the problem
lp_problem.solve()


1

In [15]:
type( costs_above_min[gen]) 

int

In [16]:
# how to creat an equeation before adding it to the problem


In [17]:

print("Status:", pulp.LpStatus[lp_problem.status])
print("Total Cost:", pulp.value(lp_problem.objective),"\n")

for per in periods:
    for gen in generators:
        print(f"x {per} - {gen}: {pulp.value(x[gen][per])}")
    print("")


for per in periods:
    for gen in generators:
        print(f"s {per} - {gen} Start-Up: {pulp.value(s[gen][per])}")
    print("")


for per in periods:
    for gen in generators:
        print(f"n {per} - {gen} Count: {pulp.value(n[gen][per])}")
    print("")

Status: Optimal
Total Cost: 988540.0 

x Period1 - Type1: 10200.0
x Period1 - Type2: 4800.0
x Period1 - Type3: 0.0

x Period2 - Type1: 16000.0
x Period2 - Type2: 14000.0
x Period2 - Type3: 0.0

x Period3 - Type1: 11000.0
x Period3 - Type2: 14000.0
x Period3 - Type3: 0.0

x Period4 - Type1: 21250.0
x Period4 - Type2: 15750.0
x Period4 - Type3: 3000.0

x Period5 - Type1: 11250.0
x Period5 - Type2: 15750.0
x Period5 - Type3: 0.0

s Period1 - Type1 Start-Up: 0.0
s Period1 - Type2 Start-Up: 0.0
s Period1 - Type3 Start-Up: 0.0

s Period2 - Type1 Start-Up: 0.0
s Period2 - Type2 Start-Up: 5.0
s Period2 - Type3 Start-Up: 0.0

s Period3 - Type1 Start-Up: 0.0
s Period3 - Type2 Start-Up: 0.0
s Period3 - Type3 Start-Up: 0.0

s Period4 - Type1 Start-Up: 0.0
s Period4 - Type2 Start-Up: 1.0
s Period4 - Type3 Start-Up: 2.0

s Period5 - Type1 Start-Up: 0.0
s Period5 - Type2 Start-Up: 0.0
s Period5 - Type3 Start-Up: 0.0

n Period1 - Type1 Count: 12.0
n Period1 - Type2 Count: 3.0
n Period1 - Type3 Count: 